<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Chat Completion for Decisions 

Decision Intelligence applied in this module:  
* Listing of various decision-making frameworks and with their descriptions by using Chat Completion  
* Decision Scenario: Recommendation of a decision-making framework for specific higher-stakes decisions (purchase of a car)  
* Illustrating a domain specific decision-making framework (millitary intelligence)  


In this module, the Semantic Kernel ability to create a chat experience will be introduced. This is a much richer experience than just sending simple prompts that are stateless and context is forgotten.

Semantic Kernel has first-class support for chat scenarios, where the user talks back and forth with the LLM, the arguments get populated with the history of the conversation. During each new run of the kernel, the arguments will be provided to the AI with content. This allows the LLM to know the historical context of the conversation.

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0

In [2]:
// Install the required NuGet packages
// Note: This also installs the Dependency Injection Package to retrieve the ChatCompletionService
#r "nuget: Microsoft.Extensions.DependencyInjection, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.26.0"

using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();

Installed Packages Microsoft.Extensions.DependencyInjection, 8.0.0 Microsoft.SemanticKernel, 1.26.0

Using Azure OpenAI Service


### Step 2 - Execute Chat Completion with Prompt Execution Settings 

> "If you can’t describe what you are doing as a process, you don’t know what you’re doing." 
>
> -- <cite>W. Edwards Deming (renowned American engineer, statistician, and management consultant)</cite>  


Using the Semantic Kernel ChatCompletion service is very similar to inovoking a prompt for basic LLM interactions. The chat completion service will provide very similar results to invoking the prompt directly.

In [3]:
// Simple prompt to list some decision frameworks this GenAI LLM is familiar with 
// LLMs are trained on a diverse range of data and can provide insights on a wide range of topics like decision frameworks
// SLMS (smaller LLMs) are trained on a more specific range of data and may not provide insights on all topics
var simpleDecisionPrompt = """
Provide a list of decision frameworks that can help improve the quality of decisions.
""";

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1000, 
    Temperature = 0.1, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

var chatResult = await chatCompletionService.GetChatMessageContentAsync(simpleDecisionPrompt, openAIPromptExecutionSettings);

Console.WriteLine(chatResult.Content);

Certainly! Here is a list of decision frameworks that can help improve the quality of decisions:

1. **SWOT Analysis**: Evaluates the Strengths, Weaknesses, Opportunities, and Threats related to a decision or organization.

2. **PEST Analysis**: Assesses the Political, Economic, Social, and Technological factors that could impact a decision.

3. **Decision Matrix (or Grid Analysis)**: Helps compare different options by scoring them against a set of criteria.

4. **Cost-Benefit Analysis**: Weighs the total expected costs against the total expected benefits of one or more actions.

5. **Pros and Cons List**: Simple method of listing the advantages and disadvantages of each option.

6. **Six Thinking Hats**: Encourages looking at a decision from multiple perspectives (logical, emotional, creative, etc.).

7. **Pareto Analysis (80/20 Rule)**: Focuses on the decisions that will have the most significant impact, based on the principle that 80% of effects come from 20% of causes.

8. **Monte 

### Step 3 - Execute Chat Completion with Streaming 

Semantic Kernel supports prompt response streaming in the ChatCompletionService. This allows responses to be streamed to the client as soon as they are made available by the LLM and API services. Below the same decision prompt is executed. However, notice that chunks are streamed and can be read by the user as soon as they appear. 

In [4]:
// Stream the chat completion response
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(simpleDecisionPrompt))
{
    Console.Write(content);
}

Certainly! Here is a list of decision frameworks that can help improve the quality of decisions:

1. **Rational Decision-Making Model**: This classic framework involves identifying the problem, gathering information, generating alternatives, evaluating alternatives, choosing the best option, and implementing the decision.

2. **SWOT Analysis**: This strategic tool assesses internal Strengths and Weaknesses, and external Opportunities and Threats to help inform decisions.

3. **Cost-Benefit Analysis**: This quantitative approach evaluates the costs and benefits of various alternatives to determine the best option.

4. **Pareto Analysis**: Also known as the 80/20 rule, this technique helps identify the most significant factors impacting a decision.

5. **Decision Matrix (Pugh Matrix, Weighted Scoring Model)**: This tool involves listing and scoring various options against a set of criteria to prioritize the best choices.

6. **Kepner-Tregoe Decision Analysis**: This structured methodolog

### Step 4 - Execute Chat Completion with Chat History 

Semantic Kernel has a ChatHistory object that can be used with the ChatCompletionService to provide historical chat context to the LLM. Notice that the ChatHistory object differentiates between the different types of chat messages:
* System Message - System or MetaPrompt. These are usually global instructions that set the "overall rules" for interacting with the LLMs.
* User Message - A message from the user
* Assistant Message - A message from the LLM. This is a message generated from an assistant or an agent. 

Identifying the messages from which role (user) it came from can help the LLM improve its own reasoning responses. This is a more sophisticated approach than passing chat history in a long dynamic string. ChatHistory objects can be serialized and persisted into databases as well.

In [13]:
// Set the overall system prompt to behave like a decision intelligence assistant (persona)
var systemPrompt = """
You are a decision intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";

// Simple instruction prompt to list 5 (five) decision frameworks this GenAI LLM is familiar with
var simpleDecisionPrompt = """
Provide five Decision Frameworks that can help improve the quality of decisions.
""";

// Create a new chat history object with proper system and user message roles
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPrompt);
chatHistory.AddUserMessage(simpleDecisionPrompt);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}
chatHistory.AddAssistantMessage(chatResponse);

Certainly! Here are five decision frameworks that can help improve the quality of decisions:

1. **Pros and Cons Analysis:**
   *Objective:* To evaluate the advantages and disadvantages of each option.
   *Steps:*
   - List all possible options.
   - For each option, list the pros (benefits) and cons (drawbacks).
   - Weigh the importance of different pros and cons.
   - Make a decision based on the net benefit of each option.

   *Pros:* Simple and intuitive, provides a balanced view.
   *Cons:* Subjective weighting can introduce bias, may overlook long-term impacts.

2. **SWOT Analysis (Strengths, Weaknesses, Opportunities, Threats):**
   *Objective:* To assess internal and external factors related to the decision.
   *Steps:*
   - List internal strengths and weaknesses of the options.
   - List external opportunities and threats that could impact the decision.
   - Analyze how strengths can be used to maximize opportunities and mitigate threats.
   - Consider how weaknesses could hi

### Step 5 - Using Chat History in a Decision Conversation 

With Semantic Kernel maintaining state with the ChatHistory object, a user can interact with the LLM in a natural way. Note the interaction below, where the user is asking which of the particular decision frameworks are best suited to purchase a car. Without the ChatHistory being passed along, the LLM would not have had the proper context to answer the question.

> "Price is what you pay. Value is what you get."  
>
> -- <cite>Warren Buffet (one of the most successful investors of all time)</cite>  

In [6]:
// Simple prompt that asks a follow-up question about the purchase of a car
// Note: No reference is made to what previous frameworks were listed.
// Note: Previous context is maintained by the Semantic Kernel ChatHistory object 
var simpleDecisionPromptFollowupQuestion = """
Which of these 5 decision frameworks would work best for making a decision to purchase a car. 
Select only the single best one.
""";

// Add User message to the chat history
chatHistory.AddUserMessage(simpleDecisionPromptFollowupQuestion);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}

// Add Assistant message to the chat history
chatHistory.AddAssistantMessage(chatResponse);

For making a decision to purchase a car, the **Decision Matrix (Weighted Scoring Model)** would likely work best. Here’s why:

### Decision Matrix (Weighted Scoring Model):

1. **Criteria Identification**: When purchasing a car, several criteria may be significant such as cost, fuel efficiency, safety features, brand reputation, maintenance costs, resale value, aesthetics, and additional features (e.g., technology, comfort).

2. **Weight Assignment**: Assign weights to each criterion based on their importance to you. For instance, if fuel efficiency is very important, it might get a high weight, while aesthetics might get a lower weight if it's less crucial.

3. **Alternative Scoring**: Evaluate each car option against the identified criteria and score them. For example, rate fuel efficiency on a scale from 1 to 10 for each car being considered.

4. **Calculation and Ranking**: Multiply each score by its respective weight, sum up the results for each car, and then rank the cars based o

> "Take time to deliberate, but when the time for action comes, stop thinking and go in."
>
> -- <cite>Napoleon Bornaparte (French Emporer and brilliant military mind)</cite>  

In [7]:
// Note: No reference is made to what previous frameworks were listed.
// Note: Previous context is maintained by the Semantic Kernel ChatHistory object 
var simpleDecisionPromptFollowupQuestionPartTwo = """
Which of these 5 decision frameworks listed above are used by the military intelligence community?
""";

// Add User message to the chat history
chatHistory.AddUserMessage(simpleDecisionPromptFollowupQuestionPartTwo);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}

// Add Assistant message to the chat history
chatHistory.AddAssistantMessage(chatResponse);

The military intelligence community employs a range of decision frameworks to analyze complex situations and make informed decisions. Among the five decision frameworks listed, the following are commonly used or adapted in various forms:

1. **SWOT Analysis**:
   - **Application**: Military intelligence uses SWOT analysis to evaluate strategic positions and assess the strengths, weaknesses, opportunities, and threats of various courses of action. This can involve analyzing the capabilities of enemy forces, the operational environment, and the potential impacts of different military strategies.

2. **Cost-Benefit Analysis (CBA)**:
   - **Application**: CBA is used for evaluating the viability and potential outcomes of military operations, investment in new technologies, and logistical decisions. It helps in determining whether the benefits of a particular action outweigh the costs in terms of resources, risks, and potential losses.

3. **Pareto Analysis (80/20 Rule)**:
   - **Applicatio

### Step 6 - Semantic Kernel inspecting Chat History 

The Semantic Kernel Chat History is a transparent object that can be inspected and written out. In fact, the ChatHistory object can be manipulated to replay chats from middle interactions to simulate different outcomes.

Execute the cell below to write out the Semantic Kernel Chat History of this chat.

In [8]:
// Print the number of chat interactions and the chat history (turns)
Console.WriteLine("Number of chat interactions: " + chatHistory.Count());

foreach (var message in chatHistory)
{
    Console.WriteLine($"# {message.Role}: {message.Content}");
}

Number of chat interactions: 7
# system: You are a decision intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
# user: Provide five Decision Frameworks that can help improve the quality of decisions.
# assistant: Certainly! Decision frameworks are structured approaches that help guide thinking and improve the quality of decisions. Here are five widely-used decision frameworks:

1. **SWOT Analysis**:
   - **Strengths**: Identify internal attributes that are helpful to achieving the objective.
   - **Weaknesses**: Identify internal attributes that are harmful to achieving the objective.
   - **Opportunities**: Identify external conditions that are helpful to achieving the objective.
   - **Threats**: Identify external conditions which could do damage to the objective.
   - **Application**: SWOT is useful for strategic planni